In [1]:
!pip install aitextgen

     |████████████████████████████████| 572 kB 600 kB/s 
     |████████████████████████████████| 87 kB 5.2 MB/s 
  Created wheel for aitextgen: filename=aitextgen-0.5.2-py3-none-any.whl size=575905 sha256=88a58b2dfafaceaa9dda4c62388866cd6b4c1a2651eb1367669731c6b208b0e0
  Stored in directory: /root/.cache/pip/wheels/83/e2/74/46c887b0989a51a7acee0c09551a3ae9d34b939fb4bea404a0
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=15b51bf835fcde6ac488093d931ff5ee38ac1bb12176a9b401cf10dca3209210
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built aitextgen fire


In [2]:
from aitextgen import aitextgen

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
ai = aitextgen(model_folder="../input/gpt-code-t1-model-v1/gpt_code_task1_model_v1",
               tokenizer_file="../input/gpt-code-t1-model-v1/gpt_code_task1_model_v1/aitextgen.tokenizer.json",
               to_gpu=True)

## Train GPT-2

Important parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM. _Unlike finetuning, since you are using a small model, you can massively increase the batch size to normalize the training_.
- **`fp16`**: Enables half-precision training for faster/more memory-efficient training. Only works on a T4 or V100 GPU.

Here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: Learning rate of the model training.


In [5]:
ai.train('../input/idl-project-code-encoder-set/encoder_docstring_to_code_train.csv',
         line_by_line=True,
         from_cache=False,
         num_steps=50_000,
         generate_every=1_000,
         save_every=1_000,
         save_gdrive=False,
         learning_rate=1e-3,
         batch_size=2,
         )

  0%|          | 0/265734 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

1,000 steps reached: saving model to /trained_model
1,000 steps reached: generating sample texts.
generate: Forward containing a topic of a topic. def topic(self, topic_id, topic_id, region):
        """
        
        """
        # Map a topic in topic in topic.
        topic_id = self.topic_id
        self.topic_id = topic_id
        self.topic_id = topic_id
        self.topic_id = topic_id
        self.topic_id = topic_id
        self.topic_id = self.topic_id
        self.topic_id = topic_id
2,000 steps reached: saving model to /trained_model
2,000 steps reached: generating sample texts.
generate: Return the status of the targets, as a string,  so the last location, or an invoked string, if the field is not found. def _get_targets(self, target_name, specification, specification_name, specification_name):
        """
        
        """
        if not isinstance(specification_name, string_types.string_types):
            raise TypeError("Unknown type '{}'".format(specification_nam


## Load a Trained Model

ai = aitextgen(model_folder="../input/finetuned-gpt/trained_model",
               tokenizer_file="../input/model-tuned/aitextgen.tokenizer.json",
               to_gpu=True)

import pandas as pd
df = pd.read_csv('../input/idl-project-code-encoder-set/encoder_docstring_to_code_test.csv')

i = 100
string = df.iloc[i]['input'][:df.iloc[i]['input'].index('def')]
code = df.iloc[i]['input'][df.iloc[i]['input'].index('def'):]

print(f'String: {string}')
print()
print(f'Code: {code}')

ai.generate(max_length = 512, 
            prompt = string, 
            top_k = 100, 
            top_p  = 0.9, 
            temperature = 0.9)